# Aufgabe 4: Gauß-Filter
Der vorgestellte Mittelwertfilter bewirkt eine Glättung des Bildes, die zur Rauschunterdrückung verwendet werden kann.
Ein anderer Faltungsoperator mit demselben Zweck ist der Gauß-Filter $A_\text{Gauß} \in \mathbb{R}^{m \times m}$.
Dabei sind die Koeffizienten der Faltungsmaske gegeben durch
\begin{equation}
 A_\text{Gauß}(h,k) = \mathrm{e}^{-\frac{h^2+k^2}{2\sigma^2}}.
\end{equation}

Die Standardabweichung $\sigma$ wird hierbei in Abhängigkeit von der Fenstergröße üblicherweise auf $\sigma = \frac{m}{5}$ gesetzt.

Berechnen Sie für verschiedene Fenstergrößen (z.B. $m \in\{3, 5, 7\}$) die Koeffizienten des Gaußfilters!
Achten Sie dabei darauf, dass die Summe der Koeffizienten auf $1$ normiert wird, damit bei der Faltung das Bild insgesamt nicht heller oder dunkler wird!
Verwenden Sie die errechneten Gaußfilter als Ersatz für den Mittelwertfilter aus der vorherigen Teilaufgabe und vergleichen Sie die Ergebnisse!

## 0. Pfade, Pakete etc.

In [ ]:
import glob
import urllib.request

%matplotlib inline
import matplotlib.pyplot as plt

import imageio
import numpy as np

In [ ]:
image_filter = 'Bilder/*.jpg'

## 1. Definition der Faltungsmaske
Definieren Sie hier zunächst die Parameter `m` und `sigma` des Filters. Berechnen Sie anschließend die Filtermaske `A_gauss` sowie (zum Vergleich) einen Mittelwertfilter derselben Größe!

In [ ]:
m = np.random.choice([3, 5, 7, 11, 21])
sigma = m / 5

offset = (m-1) // 2
A_gauss = np.asarray([
    [
        np.exp(-(((h-offset)**2) + ((k-offset)**2)) / (2*(sigma**2)))
        for h in range(m)
    ] for k in range(m)
])

A_avg = np.ones((m, m), dtype=np.float32) / (m * m)

## 2. Laden des Bildes

In [ ]:
image_path = np.random.choice(glob.glob(image_filter))
image = imageio.imread(image_path)

Für diese Aufgabe ist es wichtig, das Bild im Fließkommaformat vorliegen zu haben. Konvertieren sie `image` zu einer geeigneten Repräsentation:

In [ ]:
image = np.asarray(image, dtype=np.float32) / 255

## 3. Berechung der Faltung
Setzen Sie hier die Funktion `ex2_convolve` aus der vorherigen Aufgabe ein:

In [ ]:
def ex2_convolve(image, filter_mask):
    convolved_image = np.zeros_like(image)

    offset_y = filter_mask.shape[0] // 2
    offset_x = filter_mask.shape[1] // 2

    # Explicitly zero-pad the original image
    image_pad = np.zeros((image.shape[0] + 2 * offset_y, image.shape[1] + 2 * offset_x), dtype=image.dtype)
    image_pad[offset_y:offset_y+image.shape[0], offset_x:offset_x+image.shape[1]] = image
    
    # Convolve
    for cy in range(convolved_image.shape[0]):
        for cx in range(convolved_image.shape[1]):
            # Extract image patch of the same size as the mask centered around the current pixel,
            # multiply it element-wise with the mask, and accumulate the results.
            patch = image_pad[cy:cy+filter_mask.shape[0], cx:cx+filter_mask.shape[1]]
            convolved_image[cy, cx] = np.sum(patch * filter_mask)
    
    return convolved_image

Nun wird das gefaltete Bild mit Hilfe der Funktion berechnet:

In [ ]:
%time convolved_image_gauss = ex2_convolve(image, A_gauss)
%time convolved_image_avg = ex2_convolve(image, A_avg)

## 4. Darstellung
Stellen Sie `image`, `convolved_image_avg` und `convolved_image_gauss` nebeneinander dar:

In [ ]:
plt.figure('Convolution: image comparison', figsize=(15, 6))
plt.subplot(1,3,1, title='Original Image')
plt.imshow(image, cmap='gray', vmin=0, vmax=1)
plt.subplot(1,3,2, title='Average Filter')
plt.imshow(convolved_image_avg, cmap='gray', vmin=0, vmax=1)
plt.subplot(1,3,3, title='Gauss Filter')
plt.imshow(convolved_image_gauss, cmap='gray', vmin=0, vmax=1)
plt.show()